# Проект  №8 "Возьмёте Бэтмобиль?"
## Предсказание цены подержанного автомобиля с использованием ML и DL
Команда "Отряд Дамблдора": Борис Красницкий и Анна Журавлёва
![](https://data.whicdn.com/images/341964063/original.jpg)

В ноутбуке будет решаться задача предсказания цены подержанного автомобиля по готовым датасетам с помощью комбинации Machine Learning  и Deep Learning.

# 1. Подготовка системы и данных
### 1.1 Загружаем необходимые для работы библиотеки

In [ ]:
#!pip install -q tensorflow==2.3
#аугментации изображений
!pip install albumentations -q

#Библиотеки общего характера
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling #оценка датасета
import os
import sys
import PIL
import cv2
import re
import math
from textblob import TextBlob

# Входящие данные доступны в директории"../input/" directory, доступной только для чтения
#Загрузим библиотеки для машинного обучения и подготовки данных
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
YEAR = 2021

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Установим фиксированные условия, чтобы не повторять эти параметры в моделях

In [ ]:
rcParams['figure.figsize'] = 10, 5
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Выведем версии основных библиотек

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

### 1.2 Определим основные функции

In [ ]:
#Функция подсчета результата 
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
#Таблица сравнения результатов каждой модели
def cumulated_res(data, model,description, mape,file):
    l = len(data)
    data.loc[l]= [mape, model, description,file]
    return data

### 1.3 Импорт и обзор данных

In [ ]:
#Зададим путь
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'

#Определим датасеты из предложенных файлов
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 4, rows * 3.5))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    #ax.scatter(x = values, y = train['price'])
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna(), 
    'price': train['price'].dropna(), 
    'price_log': train['price'].apply(np.log),
})

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

Применим библиотеку pandas_profiling для описания датасетов. С помощью одной строчки кода мы получаем полную статистическую информацию по каждому признаку: тип данных, пропущенные значения, подсчет значений, нулевые значения, графические представления и корреляции. 

In [ ]:
#pandas_profiling.ProfileReport(data)

# Описание содержимого

1. bodyType - тип кузова автомобиля, категориальная переменная.
2. brand - марка автомобиля, категориальная переменная.
3. color - цвет автомобиля, категориальная переменная.
4. description - описание транспортного средства, текстовая переменная, требует обработки.
5. engineDisplacement - объем двигателя автомобиля, категориальная переменная, требует обработки.
6. enginePower - мощность двигателя автомобиля, категориальная переменная, требует обработки.
7. FuelType - тип топлива автомобиля, категориальная переменная.
8. mileage - пробег автомобиля, непрерывная переменная.
9. modelDate - год запуска производства, категориальная переменная.
10. model_info - серия автомобилей, категориальная переменная.
11. name - комбинация нескольких признаков, требующая обработки.
12. numberOfDoors - количество дверей автомобиля, категориальная переменная.
13. price - целевая переменная.
14. productionDate - год выпуска, категориальная переменная.
15. sell_id - внешний идентификатор, непрерывная переменная.
16. vehicleConfiguration - комбинация нескольких функций, требующих обработки.
17. vehicleTransmission - тип трансмиссии автомобиля, категориальная переменная.
18. Владельцы-количество владельцев, категориальная переменная.
19. Владение-время владения, требует обработки и исправления пропущенных значений.
20. ПТС - свидетельство о регистрации автомобиля, двоичная переменная.
21. Привод-компоновка автомобиля, категориальная переменная.
22. Руль-положение колеса автомобиля, категориальная переменная.

В объединенном датасете 8353 строк. В общей сложности представлены данные о 8353 автомобилях. Всего 21 переменная. Всего пропусков 5419 (или 64 % от общего количества строк), практически все пропуски в переменной 'Владение'. В рамках  проверки на наличие выбросов таковых не установлено (условно выбросами можно считать автомобили с пробегом свыше 400 тыс. км., но нами было принято решение не обрабатывать данные значения, ввиду того, что данные значения вполне могут отражать реальную ситуацию)

### 1.4 Очистка данных и получение новых признаков

In [ ]:
# Переделаем признак 'enginePower' в числовой
data['enginePower'] = data['enginePower'].apply(lambda x: int(x.split(' ')[0].replace('.', '')))

In [ ]:
# Сделаем числовым 'engineDisplacement'
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: '2.0 LTR' if x == 'undefined LTR' else x)
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: int(x.split(' ')[0].replace('.', '')))
data['engineDisplacement'].value_counts().head(3)

In [ ]:
# Сдвинем даты относительно текущего года 
data['productionDate'] = data['productionDate'].apply(lambda x: YEAR - x)
data['modelDate'] = data['modelDate'].apply(lambda x: YEAR - x)

In [ ]:
# Из числовых сделаем еще 3 новых признака:
# 'model_age' - новизна модели на момент производства с весом 0.1
data['model_age'] =  data['modelDate'] - data['productionDate']
data['model_age'] = data['model_age'].apply(lambda x: 0. if x < 0 else 0.1*x)

# 'km_pro_year' - cтепень эксплуатации(пробег/год)
data['km_pro_year'] = data['mileage']/(YEAR - data['productionDate'])

# 'mil_p' - отношение логарифма пробега к мощности
data['mil_p'] = data['mileage'].apply(lambda x: np.log(1. + x))/data['enginePower']

In [ ]:
# Признак 'Владение' вряд ли является значимым и имеет 2/3 пропусков, но 
# является интересным в плане обработки, поэтому не выбрасываем его
data['Владение'].isna().value_counts()

In [ ]:
# 'Владельцы' пока переделаем в числовой, чтобы рассчитать 'Владение',
# но позже обработаем его как категориальный
data['Владельцы'].fillna(data['Владельцы'][150], inplace=True)
data['Владельцы'] = data['Владельцы'].apply(lambda x: int(x[0]))
data['Владельцы'].value_counts()

In [ ]:
# 'Владение' в месяцах вычисляем где есть из текста, а пропущенные заменяем на
# возраст/кол-во владельцев
def own(x):    
    ''' extracts time in months from "Владение"'''
    
    try:
        pattern_year = re.compile('\d+ [гл]')
        pattern_month = re.compile('\d+ [м]')
        own_year = pattern_year.findall(x['Владение'])
        own_month = pattern_month.findall(x['Владение'])
        own_year.append('0')   # чтобы не было пустого списка
        own_month.append('0')
        return (float(own_year[0].split(' ')[0])*12 + float(own_month[0].split(' ')[0]))
    
    except:
        return x['productionDate']*12/x['Владельцы']

In [ ]:
# Применяем ф-цию 
data['Владение'] = data.apply(lambda data: own(data), axis=1)
data['Владение'].value_counts().head()

In [ ]:
# Используем текстовые признаки как категориальные без предобработки,
# за исключением 'name' и 'vehicleConfiguration' 
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 'Владельцы', 
                        'numberOfDoors', 'vehicleTransmission', 'ПТС', 'Привод', 'Руль']

# числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate', 'enginePower', 'engineDisplacement', 
                      'model_age', 'km_pro_year', 'mil_p', 'Владение']

In [ ]:
# Пропущенных значений в числовых признаках нет
data[numerical_features].isna().value_counts()

In [ ]:

# В категориальных признаках тоже нет пропущенных значений
data[categorical_features].isna().value_counts()


In [ ]:
# Посмотрим, что можно извлечь из этих признаков
data[['name', 'vehicleConfiguration']].sample(5)

In [ ]:
data['name'].value_counts()

In [ ]:
data['vehicleConfiguration'].value_counts()

In [ ]:
# Создадим признак 'allroad' для авто с полным приводом, используя данные из обоих столбцов
allroad1 = data['vehicleConfiguration'].apply(lambda x: 1 if 'ALLROAD' in x.split('_') else 0)
allroad2 = data['name'].apply(lambda x: 1 if '4WD' in x.split(' ') else 0)
data['allroad'] = (allroad1 + allroad2).apply(lambda x: 0 if x == 0 else 1)
categorical_features.append('allroad')
data['allroad'].value_counts()

In [ ]:
# Выделим AMG, Maybach и BMW-M как наиболее дорогие модели
data['lux'] = data['model_info'].apply(lambda x: 1 if (x != 'M_KLASSE') & ('M' in x) or ('8' in x) else 0)
categorical_features.append('lux')
data[data['lux'] == 1].model_info.value_counts().head()

In [ ]:
# создаем новые переменные - автомобили с большим и малым пробегом.
data['low_MtA'] = 0
data['high_MtA'] = 0

# Creating new features.
counter = 0

for MtA in data['km_pro_year']:
    if MtA < 5000:
        data.at[counter,'low_MtA'] = 1
        counter += 1
    elif MtA > 30000:
        data.at[counter,'high_MtA'] = 1
        counter += 1
    else:
        counter += 1
        
categorical_features.append('low_MtA')
categorical_features.append('high_MtA')


In [ ]:
#Внимательно посмотрим на пробег, как один из важных признаков. Создадим различные переменные в зависимости от пробега в разрезе различных моделей

mileage_by_model_info = data.groupby(['model_info'])['mileage'].agg(['mean', 'max', 'median', 'var', 'std'])
data['mean_mileage_by_model_info'] = data['model_info'].map(mileage_by_model_info['mean'])
data['max_mileage_by_model_info'] = data['model_info'].map(mileage_by_model_info['max'])
data['median_mileage_by_model_info'] = data['model_info'].map(mileage_by_model_info['median'])
data['var_mileage_by_model_info'] = data['model_info'].map(mileage_by_model_info['var'])
data['std_mileage_by_model_info'] = data['model_info'].map(mileage_by_model_info['std'])

numerical_features.append('mean_mileage_by_model_info')
numerical_features.append('max_mileage_by_model_info')
numerical_features.append('median_mileage_by_model_info')
numerical_features.append('var_mileage_by_model_info')
numerical_features.append('std_mileage_by_model_info')

In [ ]:
# заполним пропуски
data['var_mileage_by_model_info'] = data['var_mileage_by_model_info'].fillna(data['var_mileage_by_model_info'].median())
data['std_mileage_by_model_info'] = data['std_mileage_by_model_info'].fillna(data['std_mileage_by_model_info'].median())

In [ ]:
#Добавим столбец с признаком, является ли продавец дилером, что также может влиять на цену
data['dealer'] = 0
for i in range (0,8353):
    if 'AVILON' in data.description[i] or 'FAVORIT' in data.description[i] or 'РОЛЬФ' in data.description[i]:
        data['dealer'][i] = 1
categorical_features.append('dealer')


In [ ]:
#Добавим статистичекие признаки по ценам
price_by_model_info = train.groupby(['model_info'])['price'].agg(['mean', 'max', 'median', 'var', 'std'])
data['mean_price_by_model_info'] = data['model_info'].map(price_by_model_info['mean'])
data['max_price_by_model_info'] = data['model_info'].map(price_by_model_info['max'])
data['median_price_by_model_info'] = data['model_info'].map(price_by_model_info['median'])
data['var_price_by_model_info'] = data['model_info'].map(price_by_model_info['var'])
data['std_price_by_model_info'] = data['model_info'].map(price_by_model_info['std'])

numerical_features.append('mean_price_by_model_info')
numerical_features.append('max_price_by_model_info')
numerical_features.append('median_price_by_model_info')
numerical_features.append('var_price_by_model_info')
numerical_features.append('std_price_by_model_info')

In [ ]:
# заполнение пропусков
data['mean_price_by_model_info'] = data['mean_price_by_model_info'].fillna(data['mean_price_by_model_info'].median())
data['max_price_by_model_info'] = data['max_price_by_model_info'].fillna(data['max_price_by_model_info'].median())
data['median_price_by_model_info'] = data['median_price_by_model_info'].fillna(data['median_price_by_model_info'].median())
data['var_price_by_model_info'] = data['var_price_by_model_info'].fillna(data['var_price_by_model_info'].median())
data['std_price_by_model_info'] = data['std_price_by_model_info'].fillna(data['std_price_by_model_info'].median())

In [ ]:
# насколько часто встречается то или иное значение model_info. Отражает степень популярности авто
freq = data["model_info"].value_counts(normalize = True)
data["model_info_freq"] = data["model_info"].map(freq)

numerical_features.append('model_info_freq')


In [ ]:
#Добавим столбец с признаком, находится ли авто в залоге
data['залог'] = 0
for i in range (0,8353):
    if 'в залоге' in data.description[i] or 'залог' in data.description[i] or 'лизинг' in data.description[i]:
        data['залог'][i] = 1

categorical_features.append('залог')

In [ ]:
data['залог'].value_counts() # таких 154. Данный признак решено было добавить на финальном этапе выполнения проекта
# модель давала существенные ошибки, в том числе, на автомобилях, находящихся в залоге.

In [ ]:
#С помощью специальной библиотеки рассчитаем тональность текста. Предполагалось, что все описания будут носить нейтральный характер,
#  однако оценка тональности различается.
analysis = []
for i in range (0, 8353):
    analysis.append(TextBlob(data['description'][i]).sentiment[0])
data['analysis'] = analysis

numerical_features.append('analysis')


In [ ]:
#Обернём финальное преобразование признаков в функцию
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # убираем ненужные для модели признаки
    df_output.drop(['description', 'sell_id', 'name', 'vehicleConfiguration'], axis = 1, inplace=True)
    
    # Нормализация данных
    scaler = MinMaxScaler()    
    for column in numerical_features:        
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]          
       
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
           
    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values
y_log = np.log(y)    # логарифмируем целевую, это улучшает результат Catboost
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [ ]:
# Проверим их значимость
nums = pd.Series(f_regression(train_data[numerical_features], train_data['price'])[0], index = numerical_features)
nums.sort_values(inplace = True)
nums.plot(kind = 'barh')

In [ ]:
# И корреляцию
import seaborn as sns
correlation = train_data[numerical_features].corr()
plt.figure(figsize=(14, 10))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Существует сильная корреляция между объемом двигателя и мощностью. Пробег имеет сильную корреляцию с возрастными характеристиками и соотношением пробег/возраст. Возрастные характеристики коррелируют друг с другом. 

# 2. Расчёты на основе подготовленных данных
## 2.1 Maschine Learning
### 2.1.1 Модель CatboostRegressor

In [ ]:
#Подготовим датасеты для подстановки в модель
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)


In [ ]:
#Подготовим датафрейм, чтобы аккумулировать результаты по моделям и сравнивать их в единой таблице
df_cum = pd.DataFrame(columns=['test MAPE','model', 'description','submission file'])
df_cum.info()


In [ ]:
model1 = CatBoostRegressor(iterations = 8000,
                          depth=8,
                          learning_rate = 0.01,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=1000,
                          #task_type='GPU',
                         )
model1.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=500,
         use_best_model=True,
         #plot=True
         )

In [ ]:
test_predict_catboost = model1.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.4f}%")

In [ ]:
# для контроля посмотрим MAPE в обычном масштабе
test_predict_catboost_reg = np.exp(test_predict_catboost)
y_test_reg = np.exp(y_test)
print(f"TEST mape: {(mape(y_test_reg, test_predict_catboost_reg))*100:0.4f}%")

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'CatboostRegressor', 'стандарт', mape(y_test_reg, test_predict_catboost_reg)*100, 'catboost_submission.csv')

In [ ]:
sub_predict_catboost = model1.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission['price'] = np.round(sample_submission['price'].apply(lambda x: math.exp(x)).values, 0)
sample_submission.to_csv('catboost_submission.csv', index=False)

### 2.1.2 Модель GradientBoostingRegressor 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model2 = GradientBoostingRegressor(criterion='mse', # параметры подобраны с помощью GridSearchCV (здесь удалено ввиду длительности процесса)
                                     max_depth=8,
                                     min_samples_leaf=21,
                                     random_state=42,  
                                     n_estimators=4000, 
                                     max_features='sqrt',
                                     
                                     loss='huber', 
                                     learning_rate=0.026)

model2.fit(X_train, y_train)

y_train_pred =model2.predict(X_train) 

print(f"TEST mape: {(mape(y_train, y_train_pred))*100:0.2f}%")

In [ ]:
test_predict_gb = model2.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_gb))*100:0.2f}%")

In [ ]:
# для контроля посмотрим MAPE в обычном масштабе
test_predict_gb_reg = np.exp(test_predict_gb)
y_test_reg = np.exp(y_test)
print(f"TEST mape: {(mape(y_test_reg, test_predict_gb_reg))*100:0.4f}%")

# Применим VotingRegressor для улучшения результатов работы двух моделей

In [ ]:
from sklearn.ensemble import VotingRegressor
ensemble=VotingRegressor(estimators=[('GradientBoostingRegressor', model2), ('Catboost', model1)], 
                        weights=[0.5,0.5]).fit(X_train,y_train)
print('The accuracy for Grad and Catboost is:',ensemble.score(X_test,y_test))
# не получилось заставить его не выводить каждую строку на печать, поэтому вот так... 

In [ ]:
y_pred1 = ensemble.predict(X_test)
print(f"TEST mape: {(mape(y_test_reg, np.exp(y_pred1)))*100:0.2f}%")

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'ensemble', 'Grad and Catboost', mape(y_test_reg, np.exp(y_pred1))*100, 'ensemble_submission.csv')

In [ ]:
# анализ результата работы модели

#plt.scatter(test_predict_catboost_reg,  y_train_pred - y_train,
#            c='blue', marker='o', label='Training data')
plt.scatter(np.exp(y_pred1),  np.exp(y_pred1) - y_test_reg,
            c='blue', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=15000000, lw=2, color='red')
plt.xlim([-10, 15000000])
plt.tight_layout()

как видно на графике - модель дает существенные ошибки на автомобилях с ценой свыше 6 млн рублей

In [ ]:
# проверим количество ошибок и их масштаб
X_train['price_predict'] = np.exp(y_train_pred)
X_train ['loss'] = np.exp(y_train) - X_train.price_predict

In [ ]:
X_train ['loss']

In [ ]:
X_train[(X_train.loss == 0)].index # нет ни одного точного попадания...

In [ ]:
X_train[(X_train.loss > -1000)&(X_train.loss < 1000)] # 84 ответа с ошибкой от -1000 до 1000 рублей. 

In [ ]:
X_train[(X_train.loss > X_train.loss.quantile(0.9))].index # по индексу максимальных ошибок можно ознакомиться с авто и их описанием в data



In [ ]:
sub_predict_ensemble = ensemble.predict(X_sub)
sample_submission['price'] = np.exp(sub_predict_ensemble)
sample_submission.to_csv('ensemble_submission.csv', index=False)

## 2.2 Подключаем Deep Learning
### 2.2.1 Делаем нейросеть на табличных данных

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)


In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=20, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=1000, # фактически мы обучаем пока EarlyStopping не остановит обучение. 
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
#Сохраняем веса обученной модели
model.save('../working/nn_1.hdf5')


In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
# анализ результата работы модели

#plt.scatter(test_predict_catboost_reg,  y_train_pred - y_train,
#            c='blue', marker='o', label='Training data')
plt.scatter(test_predict_nn1[:,0],  test_predict_nn1[:,0] - y_test,
            c='blue', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=15000000, lw=2, color='red')
plt.xlim([-10, 15000000])
plt.tight_layout()

По-прежнему существенные ошибки на авто дороже 6 млн.

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Neuro Net 1', 'табличная', mape(y_test, test_predict_nn1[:,0])*100, 'nn1_submission.csv')

In [ ]:
# Смешаем нейросеть и ensemble, проверим МAPE.
blend_predict1 = (np.exp(y_pred1)*0.7 + test_predict_nn1[:,0]*0.3) 
print(f"TEST mape: {(mape(y_test, blend_predict1))*100:0.2f}%")

In [ ]:
blend_sub_predict1 = (sub_predict_ensemble*0.7 + sub_predict_nn1[:,0]*0.3) 
sample_submission['price'] = blend_sub_predict1
sample_submission.to_csv('blend_submission_nn1.csv', index=False)

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Blend 1', 'табличная + ensemble', mape(y_test, blend_predict1)*100, 'blend_submission_nn1.csv')

### 2.2.2 Подключим в модель блок NLP

In [ ]:
# В описаниях находим 5 пустых значений и заменяем их на подобные
data['description_1'] = data['description'].apply(lambda x: len(x))
data[data['description_1'] == 1]['description']

In [ ]:
data['description'].loc[388] = data['description'].loc[7069]
data['description'].loc[1005] = data['description'].loc[4753]
data['description'].loc[4265] = data['description'].loc[7448]
data['description'].loc[6381] = data['description'].loc[3004]
data['description'].loc[8283] = data['description'].loc[8349]

In [ ]:
# Добавим для информативности в описания данные из других колонок
data.description = data.brand + ' ' + data.bodyType + ' ' +\
                   data.enginePower.apply(lambda x: str(x)) +  ' ' + 'лс' + ' ' +\
                   data.productionDate.apply(lambda x: str(x)) + ' ' + 'год' + ' ' + data.description

In [ ]:
data['description']

In [ ]:
# Сделаем лемматизацию и уберем стоп-слова из текстов объявлений
!pip install pymystem3

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian") # удаляет предлоги союзы и пр.

# Исключим из стоп-листа 'не'
operators = set(('не', 'не'))
russian_stopwords = set(russian_stopwords) - operators

def lemma_text(text):
        
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

#Example    
lemma_text("Машина тут продаётся не в первый раз, НО!")

In [ ]:
# Благодаря лемматизации кол-во токенов в словаре tokenize.word_index
# уменьшается с 39591 до 22329
data.description = data.description.apply(lambda x: lemma_text(x))
data.description[10]


In [ ]:
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-•―☛。«❤️✅✔₽➥]"

for i in range (1,8353):
    data['description'][i] = data['description'][i].replace (patterns, '')

In [ ]:
# TOKENIZER
# The maximum number of words to be used
MAX_WORDS = 50000 # 50000
# Max number of words in each complaint
MAX_SEQUENCE_LENGTH = 256 # 256

In [ ]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [ ]:
len(tokenize.word_index)

In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH, name='nlp1'))
model_nlp.add(L.GRU(256, return_sequences=True, name='nlp2'))# в процессе выполнения проекта использовались и LSTM слои и GRU. GRU показал результат лучше
model_nlp.add(L.Dropout(0.5)) # также пробовали различные вариации. Выбрано такое значение ввиду переобучения модели.
model_nlp.add(L.GRU(128, name='nlp3'))# LSTM # 256 и 128
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu", name='nlp4'))
model_nlp.add(L.Dropout(0.25))

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu", name='mlp1'))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu", name='mlp2'))
model_mlp.add(L.Dropout(0.5))

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output], name='comb1')
# being our regression head
head = L.Dense(256, activation="relu", name='comb2')(combinedInput)
head = L.Dense(1, activation="linear", name='comb3')(head)

model_1 = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head, name='model_1')

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model_1.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=20, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model_1.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение.
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model_1.save('../working/nn_mlp_nlp.hdf5')


In [ ]:
test_predict_nn2 = model_1.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
# анализ результата работы модели

#plt.scatter(test_predict_catboost_reg,  y_train_pred - y_train,
#            c='blue', marker='o', label='Training data')
plt.scatter(test_predict_nn2[:,0],  test_predict_nn2[:,0] - y_test,
            c='blue', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=15000000, lw=2, color='red')
plt.xlim([-10, 15000000])
plt.tight_layout()

Все также 

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Neuro Net 2', 'табличная + NLP', mape(y_test, test_predict_nn2[:,0])*100, 'nn2_submission.csv')

In [ ]:
# Смешаем нейросеть и ensemble, проверим МAPE.
blend_predict2 = (np.exp(y_pred1)*0.7 + test_predict_nn2[:,0]*0.3) 
print(f"TEST mape: {(mape(y_test, blend_predict2))*100:0.2f}%")

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Blend 2', 'ensemble + NLP', mape(y_test, blend_predict2)*100, 'blend_submission_nn2.csv')

In [ ]:
sub_predict_nn2 = model_1.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

### 2.2.3 Добавим в нейросеть фото

In [ ]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [ ]:
# Для аугментации используем Albumentations
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, RandomShadow, 
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

augmentation = Compose([
    HorizontalFlip(p=0.4),
    OneOf([
        RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15),
        RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05)
    ],p=0.3),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.45),
    Blur(p=0.15),
    HueSaturationValue(p=0.25),
    #RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, always_apply=False, p=0.25),
])

#пример
plt.figure(figsize = (12,8))
for i in range(12):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 4, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

In [ ]:
# Используем предобученную EfficientNetB6. Модель выбрана перебором
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB6(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
#строим нейросеть для анализа табличных данных. В ходе выполнения проекта пытались подавать 
# на вход уже обученную на предыдущих этапах модель но показатели были хуже
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    ])

In [ ]:
# NLP. В ходе выполнения проекта пытались подавать 
# на вход уже обученную на предыдущих этапах модель но показатели были хуже
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.GRU(256, return_sequences=True),
    L.Dropout(0.5),
    L.GRU(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
model.summary()

# Fine tuning

In [ ]:
# Не тренируем часть с картинками
efficientnet_model.trainable = False

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')# best_model
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=20,#Было 20
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
efficientnet_model.trainable = True

# разморозим половину слоев модели
fine_tune_at = len(efficientnet_model.layers)//2


for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False
len(efficientnet_model.trainable_variables)

In [ ]:
model.load_weights('../working/best_model.hdf5')

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=20,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
efficientnet_model.trainable = True #разморозим все слои 
len(efficientnet_model.trainable_variables)

In [ ]:
model.load_weights('../working/best_model.hdf5')

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit(train_dataset.batch(20),
                    epochs=5,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
# анализ результата работы модели

#plt.scatter(test_predict_catboost_reg,  y_train_pred - y_train,
#            c='blue', marker='o', label='Training data')
plt.scatter(test_predict_nn3[:,0],  test_predict_nn3[:,0] - y_test,
            c='blue', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=15000000, lw=2, color='red')
plt.xlim([-10, 15000000])
plt.tight_layout()

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Neuro Net 3', 'табличная + NLP + фото', mape(y_test, test_predict_nn3[:,0])*100, 'nn3_submission.csv')

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = np.round(sub_predict_nn3[:,0], 0)
sample_submission.to_csv('nn3_submission.csv', index=False)
sample_submission.head(3)

In [ ]:
blend_predict = (np.exp(y_pred1)*0.8+ test_predict_nn3[:,0]*0.2)
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
blend_predict = (np.exp(y_pred1)*0.8+ test_predict_nn3[:,0]*0.1 + test_predict_nn2[:,0]*0.1)
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Blend 2', 'nn3 + catboost + веса', (mape(y_test, blend_predict))*100, 'blend_submission2.csv')

In [ ]:
blend_sub_predict = np.round((np.exp(sub_predict_ensemble)*0.8 + sub_predict_nn3[:,0]*0.2), 0)
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission2.csv', index=False)

In [ ]:
blend_sub_predict = np.round((np.exp(sub_predict_ensemble)+ np.round(sub_predict_nn3[:,0], 0))/2, 0)
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission3.csv', index=False)

In [ ]:
#Добавим расчет в сводную таблицу
cumulated_res(df_cum, 'Blend 3', 'nn3 + catboost', '?', 'blend_submission3.csv')

# Краткое изложение и выводы¶
В рамках проекта было сделано следующее:
- инициализированы необходимые библиотеки, заданы условия визуализации и загружен набор данных.
- мы провели EDA, очистили и обработали табличную информацию.
- обучили 2 модели - CatBoost и Gradienboosting с последующим применением votingregressor для усреднения результатов их работы.  
- обучена NN на табличных данных, улучшен результат с помощью подбора гиперпараметров.
- лемматизировали тексты, очистили их от символов и обучили НЛП на основе полученных данных.
- произвели аугментацию изображений и обучили NN, улучшив ее результат с помощью точной настройки.
- использовали fine tuning.
- использовали смешивание для усреднения результатов, в том числе, с применением весов.
- проведена последовательная оценка работы моделей с предметным анализом их ошибок.

В результате получилось улучшить результат относительно baseline на 1,86519 (итоговый результат на Kaggle = 10.84210, 8 место).

Из полученных результатов можно сделать следующие выводы:

Выполненная работа позволяет прогнозировать стоимость автобилей со средней ошибкой в 10.8 %. По нашему мнению эта ошибка значительна, но при проведении анализа работы моделей было установлено, что, основной вклад в усредненную ошибку вносят нестандартные модели авто. Так, например, в нашем датасете присутствуют бронированные автомобили. Кроме того, цена выставляемого на продажу автомобиля не всегда соответствует его реальной цене, зачастую она завышена.

Что планировалось сделать, но не успели:
- увеличение изображений (на предыдущем проекте давало неплохой прирост качества)
- ТТА
- также необходимо более детально работать с текстовой частью